In [5]:
import PIL.Image as Image
import torchvision.transforms as transforms
import torch
from torchvision import models
import torch.nn as nn
import torch
import torchvision.models as models
from executorch.backends.xnnpack.partition.xnnpack_partitioner import XnnpackPartitioner
from executorch.exir import to_edge_transform_and_lower


def test_input(model: torch.nn.Module, img_path: str = ""):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                            std=[0.229, 0.224, 0.225])
    ])
    # 추론
    if img_path == "":
        random_input = torch.randn(1, 3, 224, 224).to(device)

        with torch.no_grad():
            output = nn.functional.softmax(model(random_input), dim=1)
    else:
                # 이미지 열기
        img = Image.open(img_path).convert("RGB")
        img_tensor = preprocess(img).unsqueeze(0).to(device)
        with torch.no_grad():
            output = nn.functional.softmax(model(img_tensor), dim=1)

    print(output)


def export_model_to_pte(model: torch.nn.Module, save_path: str):
    sample_inputs = (torch.randn(1, 3, 224, 224), )
    model = model.eval().to("cpu")

    et_program = to_edge_transform_and_lower(
        torch.export.export(model, sample_inputs),
        partitioner=[XnnpackPartitioner()]
    ).to_executorch()

    with open(save_path, "wb") as f:
        f.write(et_program.buffer)
    print(f"Model exported to {save_path}")

In [6]:
class_names = ['disease','downy', 'healthy', 'powdery']
healthy_img = 'good_img.jpeg'

# ==============================================================
# 2. 모델 정의
# ==============================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

mobilenet = models.mobilenet_v2(pretrained=True)
# 출력층의 뉴런 개수를 데이터셋의 클래스 개수에 맞춰 자동으로 설정
mobilenet.classifier[1] = nn.Linear(mobilenet.last_channel, len(class_names))
mobilenet = mobilenet.eval().to(device)

mobilenet.load_state_dict(torch.load('/Cucumber-Disease-Prediction/AI/to_mobile/mob_ood_nc.pth', weights_only=False, map_location=device))
mobilenet_pte_name = "mobilenet_v2_cucumber_disease.pte"

In [4]:
export_model_to_pte(mobilenet, mobilenet_pte_name)

In [1]:
from ultralytics import YOLO


yolo_model = YOLO("/home/eden/Documents/JNU/2025-2/Industry-University-Cooperation-Project/Cucumber-Disease-Prediction/AI/YOLO/best.pt")
yolo_model.export(format="tflite", imgsz=224)

Ultralytics 8.3.229 🚀 Python-3.12.3 torch-2.8.0+cu128 CPU (12th Gen Intel Core i7-12700F)
YOLOv12n summary (fused): 159 layers, 2,557,313 parameters, 0 gradients, 6.3 GFLOPs

PyTorch: starting from '/home/eden/Documents/JNU/2025-2/Industry-University-Cooperation-Project/Cucumber-Disease-Prediction/AI/YOLO/best.pt' with input shape (1, 3, 224, 224) BCHW and output shape(s) (1, 7, 1029) (5.2 MB)


E0000 00:00:1763535767.999656  154745 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763535768.003179  154745 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763535768.012611  154745 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763535768.012626  154745 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763535768.012627  154745 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763535768.012628  154745 computation_placer.cc:177] computation placer already registered. Please check linka


TensorFlow SavedModel: starting export with tensorflow 2.19.1...

ONNX: starting export with onnx 1.19.1 opset 22...
ONNX: slimming with onnxslim 0.1.74...
ONNX: export success ✅ 1.2s, saved as '/home/eden/Documents/JNU/2025-2/Industry-University-Cooperation-Project/Cucumber-Disease-Prediction/AI/YOLO/best.onnx' (10.0 MB)
TensorFlow SavedModel: starting TFLite export with onnx2tf 1.28.3...
Saved artifact at '/home/eden/Documents/JNU/2025-2/Industry-University-Cooperation-Project/Cucumber-Disease-Prediction/AI/YOLO/best_saved_model'. The following endpoints are available:

* Endpoint 'serving_default'
  inputs_0 (POSITIONAL_ONLY): TensorSpec(shape=(1, 224, 224, 3), dtype=tf.float32, name='images')
Output Type:
  TensorSpec(shape=(1, 7, 1029), dtype=tf.float32, name=None)
Captures:
  131180704791056: TensorSpec(shape=(4, 2), dtype=tf.int32, name=None)
  131180704790480: TensorSpec(shape=(3, 3, 3, 16), dtype=tf.float32, name=None)
  131180704792016: TensorSpec(shape=(16,), dtype=tf.float

I0000 00:00:1763535783.544704  154745 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00:1763535783.544783  154745 single_machine.cc:374] Starting new session
W0000 00:00:1763535784.090114  154745 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1763535784.090134  154745 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
I0000 00:00:1763535784.634810  154745 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00:1763535784.634882  154745 single_machine.cc:374] Starting new session
W0000 00:00:1763535785.115624  154745 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1763535785.115638  154745 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.


TensorFlow SavedModel: export success ✅ 17.9s, saved as '/home/eden/Documents/JNU/2025-2/Industry-University-Cooperation-Project/Cucumber-Disease-Prediction/AI/YOLO/best_saved_model' (25.2 MB)

TensorFlow Lite: starting export with tensorflow 2.19.1...
TensorFlow Lite: export success ✅ 0.0s, saved as '/home/eden/Documents/JNU/2025-2/Industry-University-Cooperation-Project/Cucumber-Disease-Prediction/AI/YOLO/best_saved_model/best_float32.tflite' (10.0 MB)

Export complete (18.0s)
Results saved to /Cucumber-Disease-Prediction/AI/YOLO
Predict:         yolo predict task=detect model=/home/eden/Documents/JNU/2025-2/Industry-University-Cooperation-Project/Cucumber-Disease-Prediction/AI/YOLO/best_saved_model/best_float32.tflite imgsz=224  
Validate:        yolo val task=detect model=/home/eden/Documents/JNU/2025-2/Industry-University-Cooperation-Project/Cucumber-Disease-Prediction/AI/YOLO/best_saved_model/best_float32.tflite imgsz=224 data=D:/jiseong/cucumber/dataset/data.yaml  
Visualize:   

'/home/eden/Documents/JNU/2025-2/Industry-University-Cooperation-Project/Cucumber-Disease-Prediction/AI/YOLO/best_saved_model/best_float32.tflite'

In [12]:
from pathlib import Path

import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image

from executorch.runtime import Runtime, Verification


# ---------------------- 공통 전처리 (원래 코드와 동일) ----------------------
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


def run_pytorch_model(model: torch.nn.Module, img_path: str | None = None):
    """PyTorch eager 모델에서 출력 얻기"""
    device = torch.device("cpu")
    model = model.eval().to(device)

    if img_path is None or img_path == "":
        x = torch.randn(1, 3, 224, 224, device=device)
    else:
        img = Image.open(img_path).convert("RGB")
        x = preprocess(img).unsqueeze(0).to(device)
        print(x)

    with torch.no_grad():
        out = model(x)
        out = nn.functional.softmax(out, dim=1)

    return x.cpu(), out.cpu()   # 입력과 출력을 같이 리턴


def run_pte_model(pte_path: str, input_tensor: torch.Tensor):
    """
    ExecuTorch .pte를 로드해서 같은 입력으로 추론 실행.
    input_tensor: shape (1,3,224,224) on CPU
    """
    # ExecuTorch Runtime 로드
    et_runtime = Runtime.get()
    program = et_runtime.load_program(
        Path(pte_path),
        verification=Verification.Minimal,  # 간단한 검증
    )
    print("PTE program methods:", program.method_names)

    # 보통 이름이 "forward" 하나일 것임
    method = program.load_method("forward")

    # ExecuTorch는 tuple(inputs) 을 받음
    inputs = (input_tensor.contiguous(),)
    outputs = method.execute(inputs)  # list[Tensor] 형태로 반환됨

    # 대부분 분류 모델이므로 첫번째 텐서를 사용
    et_out = outputs[0]
    et_out = nn.functional.softmax(et_out, dim=1)

    return et_out.cpu()


def verify_pte(model: torch.nn.Module, pte_path: str, img_path: str = ""):
    """
    1) PyTorch eager 모델과
    2) ExecuTorch .pte
    를 같은 입력으로 돌려서 차이를 비교하는 함수.
    """
    # 1. PyTorch 쪽 출력
    x, ref_out = run_pytorch_model(model, img_path)

    # 2. PTE 쪽 출력
    pte_out = run_pte_model(pte_path, x)

    # 3. 차이 비교
    if ref_out.shape != pte_out.shape:
        print(f"[WARN] Output shape mismatch: PyTorch {ref_out.shape}, PTE {pte_out.shape}")
    else:
        diff = (ref_out - pte_out).abs()
        max_diff = diff.max().item()
        mean_diff = diff.mean().item()
        print(f"[INFO] max |Δ| = {max_diff:.6f}, mean |Δ| = {mean_diff:.6f}")

    # 일부 값 출력
    print("\n[PyTorch output (first 5 probs)]")
    print(ref_out[0, :5])
    print("\n[PTE output (first 5 probs)]")
    print(pte_out[0, :5])

    return ref_out, pte_out

mobilenet_pte_name = '/home/eden/Documents/JNU/2025-2/Industry-University-Cooperation-Project/Cucumber-Disease-Prediction/AI/to_mobile/mobilenet_v2_cucumber_disease.pte'
test_img  = '/home/eden/Documents/JNU/2025-2/Industry-University-Cooperation-Project/Cucumber-Disease-Prediction/image copy.png'

verify_pte(mobilenet, mobilenet_pte_name, test_img)

tensor([[[[ 0.6049,  0.6392,  0.6392,  ...,  1.5297,  1.5468,  1.4954],
          [ 0.6221,  0.6392,  0.6563,  ...,  1.4783,  1.4783,  1.4440],
          [ 0.6221,  0.6221,  0.6221,  ...,  1.4612,  1.4440,  1.4269],
          ...,
          [ 0.4508,  0.3823,  0.3823,  ...,  0.6049,  0.4508,  0.3138],
          [ 0.6392,  0.6221,  0.6221,  ...,  0.4508,  0.4337,  0.2796],
          [ 0.9646,  0.9988,  0.9817,  ...,  0.0741,  0.2796,  0.4166]],

         [[ 0.6779,  0.6954,  0.6954,  ...,  1.8508,  1.8158,  1.7458],
          [ 0.6779,  0.6779,  0.6779,  ...,  1.7808,  1.7458,  1.6933],
          [ 0.6779,  0.6604,  0.6429,  ...,  1.7283,  1.7108,  1.6933],
          ...,
          [ 0.2927,  0.1527,  0.0826,  ...,  0.7829,  0.5553,  0.4328],
          [ 0.3277,  0.2577,  0.2577,  ...,  0.7304,  0.6078,  0.4503],
          [ 0.6604,  0.6604,  0.7129,  ...,  0.5378,  0.6254,  0.6604]],

         [[ 0.5311,  0.5659,  0.5834,  ...,  2.0300,  1.9951,  1.9254],
          [ 0.5311,  0.5659,  

(tensor([[0.0672, 0.8615, 0.0320, 0.0393]]),
 tensor([[0.0672, 0.8615, 0.0320, 0.0393]]))